<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/functionalcpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ
1. C++で関数型プログラミング風に書きたい、と思った。
1. functional programming in C++ という本のchapter01がネットで読めるので、導入だけでもいいと思って読んでいるところ。
1. https://itbook.store/files/9781617293818/chapter1.pdf

In [24]:
# 最初にでてくる、関数型でない書き方の例を、とりあえず動かす。
%%writefile countlines01.cpp
#include <iostream>
#include <fstream>
#include <vector>
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    char c = 0;
    for (const auto& file : files) {
        int line_count = 0;
        std::ifstream in(file);
        while (in.get(c)) {
            if (c == '\n') {
                line_count++;
            }
        }
        results.push_back(line_count);
    }
    return results;
}
int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines01.cpp


In [25]:
!g++ countlines01.cpp -o countlines01; ./countlines01

29


In [21]:
!wc countlines01.cpp

 29  75 702 countlines01.cpp


行数が一致することを確認。

In [28]:
# std::count アルゴリズムを使う
%%writefile countlines02.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}
std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results;
    for (const auto& file : files) {
        results.push_back(count_lines(file));
    }
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines02.cpp


In [29]:
!g++ countlines02.cpp -o countlines02; ./countlines02

29


In [36]:
%%writefile countlines03.cpp
#include <algorithm>
#include <fstream>
#include <iostream>
#include <vector>

int count_lines(const std::string& filename) {
    std::ifstream in(filename);
    return std::count(std::istreambuf_iterator<char>(in),
                      std::istreambuf_iterator<char>(), '\n');
}

std::vector<int> count_lines_in_files(const std::vector<std::string>& files) {
    std::vector<int> results(files.size());
    std::transform(files.cbegin(), files.cend(), results.begin(), count_lines);
    return results;
}

int main() {
    std::vector<std::string> files{"countlines01.cpp"};
    std::vector<int> results;

    results = count_lines_in_files(files);

    for (int i = 0; i < results.size(); i++) {
        std::cout << results[0] << "\n";
    }

    return 0;
}


Overwriting countlines03.cpp


In [37]:
!g++ countlines03.cpp -o countlines03; ./countlines03

29


In [ ]:
%%writefile countlines04.cpp

In [ ]:
!g++ countlines04.cpp -o countlines04; ./countlines04